In [1]:
%set_env ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}%set_env LANGSMITH_TRACING=true%set_env LANGSMITH_API_KEY=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%set_env TAVILY_API_KEY=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%set_env OPENAI_API_KEY=${OPENAI_API_KEY}%set_env LANGCHAIN_API_KEY=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%set_env LANGCHAIN_TRACING_V2=true%set_env LANGCHAIN_PROJECT="Multi-agent Collaboration"

env: ANTHROPIC_API_KEY=${ANTHROPIC_API_KEY}
env: LANGSMITH_TRACING=true
env: LANGSMITH_API_KEY=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
env: TAVILY_API_KEY=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
env: OPENAI_API_KEY=${OPENAI_API_KEY}
env: LANGCHAIN_API_KEY=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
env: LANGCHAIN_TRACING_V2=true
env: LANGCHAIN_PROJECT="Multi-agent Collaboration"


In [2]:
import config# from langchain.chat_models import ChatOpenAIfrom langchain_openai import ChatOpenAIfrom langchain_anthropic import ChatAnthropicfrom langchain.schema import AIMessage, HumanMessage, SystemMessageimport pandas as pdimport pandasql as psqldef ChatAnthropicSKT(model="anthropic/claude-3-5-sonnet-20240620"):    llm_api_key = config.CUSTOM_API_KEY"https://api.platform.a15t.com/v1"    # model = "anthropic/claude-3-5-sonnet-20240620"    model = ChatOpenAI(        temperature=0,          openai_api_key=llm_api_key,         openai_api_base=llm_api_url,         model=model        )    return modelllm = ChatAnthropicSKT()# llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")# llm = ChatOpenAI(model="gpt-3.5-turbo")# llm = ChatAnthropicSKT(model="azure/openai/gpt-4-1106")

In [3]:
import json
from typing import Annotated, List

from langchain_core.tools import tool

from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional

from typing_extensions import TypedDict

@tool
def check_schema(
    # dataframe_json: Annotated[str, "dataframe in JSON format."], 
    dataframe_path: Annotated[str, "path to the dataframe."]
):    
    """
    Check the schema of a dataframe.
    
    Args:
        # dataframe_json (str): The dataframe in JSON format.
        dataframe_path (str): The path to the dataframe.
    
    Returns:
        str: The schema of the dataframe.
    """

    # print("=====check_schema>dataframe_json: ",dataframe_json)
    print("=====check_schema>dataframe_path: ",dataframe_path)

    try:
        # If it's a string, try to parse it as JSON
        # try:
        #     if isinstance(dataframe_json, str):
        #         json_data = json.loads(dataframe_json)
        #     else:
        #         json_data = dataframe_json
            
        #     # Create a DataFrame
        #     df = pd.DataFrame(json_data)
        # except:
        #     df = pd.read_csv(dataframe_path)

        df = pd.read_csv(dataframe_path)
        
        # Get the schema
        schema = df.dtypes.to_string()
        return f"DataFrame schema:\n{schema}"
    except json.JSONDecodeError as e:
        error_msg = f"Error decoding JSON string: {str(e)}"
        return error_msg
    except Exception as e:
        error_msg = f"Error checking schema: {str(e)}"
        return error_msg

@tool
def generate_sql_query(
    user_request: Annotated[str,"user request to generate a sql query"],
    dataframe_schema: Annotated[str,"dataframe schema to generate a sql query"],
    dataframe_name: Annotated[str,"dataframe name to search"]
) -> str:
    """
    A tool to generate a SQL query based on the user request and schema in the message.
    """

    # print("=====generate_sql_query>dataframe_schema: ",dataframe_schema)
    # print("=====generate_sql_query>dataframe_name: ",dataframe_name)

    try:
        prompt = f"Given the following user request:'{user_request}' and schema:\n{dataframe_schema} and table name:'{dataframe_name}'\n\nGenerate a SQL query for the following message."
        sql_query = llm.invoke([HumanMessage(content=prompt)]).content
        return sql_query
    except Exception as e:
        return ""

# @tool
# def search_dataframe(
#     dataframe_json: Annotated[str, "dataframe in JSON format."], 
#     sql_query: Annotated[str, "SQL query to execute."]
# ) -> str:
#     """
#     A tool to search a dataframe using a SQL query from SQL_Expert.
#     """
#     print("=====search_dataframe>dataframe_json: ",dataframe_json)
#     print("=====search_dataframe>sql_query: ",sql_query)
#     try:
#         df = pd.read_json(dataframe_json)
#         result = psql.sqldf(sql_query, locals())
#         return result.to_json(orient='records')
#     except Exception as e:
#         return "[]"

@tool
def search_dataframe(
    dataframe_path: Annotated[str, "path to the dataframe."],
    sql_query: Annotated[str, "SQL query to execute."]
) -> str:
    """
    A tool to search a dataframe using a SQL query from SQL_Expert.
    """
    # print("=====search_dataframe>dataframe_path: ",dataframe_path)
    print("=====search_dataframe>sql_query: ",sql_query)
    try:
        df = pd.read_csv(dataframe_path)
        # print("=====search_dataframe>result: ",stock_data_krx)
        result = psql.sqldf(sql_query, locals())
        return result.to_json(orient='records')
    except Exception as e:
        return "[]"

In [4]:
from typing import List, Optional, Dict, Any, Annotated
import functools
import operator

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph, START

from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)

def create_agent(
    llm,
    tools: list,
    system_message: str,
) -> str:

    tool_names = [tool.name for tool in tools]
    tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

    """Create a function-calling agent and add it to the graph."""
    
    system_prompt = "You are a helpful AI assistant, collaborating with other assistants."
    " Use the provided tools to progress towards answering the question."
    " If you are unable to fully answer, that's OK, another assistant with different tools "
    " will help where you left off. Execute what you can to make progress."
    " If you or any of the other assistants have the final answer or deliverable,"
    " prefix your response with FINAL ANSWER so the team knows to stop."
    " You have access to the following tools: \n{tool_descriptions}.\n{system_message}"

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )

    prompt = prompt.partial(system_message=system_message)
    # prompt = prompt.partial(tool_names=tool_names)
    prompt = prompt.partial(tool_descriptions=tool_descriptions)

    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

# def create_agent(llm, tools: list, system_message: str):
#     """Create an agent."""

#     tool_names = [tool.name for tool in tools]
#     tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

#     prompt = ChatPromptTemplate.from_messages(
#         [
#             (
#                 "system",
#                 "You are a helpful AI assistant, collaborating with other assistants."
#                 " Use the provided tools to progress towards answering the question."
#                 " If you are unable to fully answer, that's OK, another assistant with different tools "
#                 " will help where you left off. Execute what you can to make progress."
#                 " If you or any of the other assistants have the final answer or deliverable,"
#                 " prefix your response with FINAL ANSWER so the team knows to stop."
#                 " You have access to the following tools: {tool_names}\n{tool_descriptions}.\n{system_message}",
#             ),
#             MessagesPlaceholder(variable_name="messages"),
#         ]
#     )

#     prompt = prompt.partial(system_message=system_message)
#     prompt = prompt.partial(tool_names=tool_names)
#     prompt = prompt.partial(tool_descriptions=tool_descriptions)
#     return prompt | llm.bind_tools(tools)

def agent_node(state, agent, name):

    # print(f"=====1. agent_node>new_state({name}): ",state['messages'][-1].content)

    state_tmp = state.copy()

    state_tmp["messages"] = [HumanMessage(content=state_tmp['messages'][-1].content+"\n\n[Current Available Arguments and Values for Tools]\n"+"\n".join(["\t "+k+": "+str(v) for k, v in state_tmp.items() if k not in ['messages','team_members','next','agent_result']]
)
# +"\n\n"+state_tmp["agent_result"]
, name=name)]
    
    # print(f"=====2. agent_node>new_state({name}): ",state_tmp['messages'][-1].content)

    result = agent.invoke(state_tmp)
    output = result["output"]

    new_state = state.copy()

    new_state["agent_result"] = new_state["agent_result"]  + f"""\n\n{len(state_tmp["messages"])}:\n""" + output
    new_state["messages"] = new_state["messages"] + [HumanMessage(content=output, name=name)]

    # print(f"=====3. agent_node>new_state({name}): ",output)

    # return {"messages": [HumanMessage(content="END", name=name)]}    
    return new_state

schema_agent = create_agent(llm, [check_schema], "You are a data analyst specializing in understanding data schemas. Use the provided tools to analyze the schema of the given data. Just identify the schema and Do not generate SQL queries.")
schema_node = functools.partial(agent_node, agent=schema_agent, name="Schema_Analyst")

sql_agent = create_agent(llm, [generate_sql_query], "You are a SQL expert who generates queries based on natural language requests and data schemas. Use the provided tools your expertise to generate SQL queries. Remark that all table names for SQL queries shoudld be 'df'. If you have the request with SQL queries already, you can skip your turn.")
sql_node = functools.partial(agent_node, agent=sql_agent, name="SQL_Expert")

search_agent = create_agent(llm, [search_dataframe], "You are a data scientist who executes or runs SQL queries on pandas DataFrames and interprets the results. Use the provided tools to search the DataFrame. If you have given SQL queries, run them on the DataFrame.")
search_node = functools.partial(agent_node, agent=search_agent, name="Search_Agent")

data_scientist = create_agent(llm, [check_schema, generate_sql_query, search_dataframe], "You are a data analyst specializing in understanding data schemas. You are a SQL expert who generates queries based on natural language requests and data schemas. You are a data scientist who executes SQL queries on pandas DataFrames and interprets the results")
data_scientist_node = functools.partial(agent_node, agent=data_scientist, name="Data_Scientist")


In [5]:
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: Optional[str]
    # dataframe_json: str
    dataframe_path: str
    table_name: Optional[str]
    # sql_query: Optional[str]
    # search_result: Optional[str]
    agent_result: Optional[str]

In [6]:
def create_team_supervisor(llm: ChatOpenAI, system_prompt, members) -> str:    """An LLM-based router."""    options = ["FINISH"] + members    function_def = {        "name": "route",        "description": "Select the next role.",        "parameters": {            "title": "routeSchema",            "type": "object",            "properties": {                "next": {                    "title": "Next",                    "anyOf": [                        {"enum": options},                    ],                },            },            "required": ["next"],        },    }    prompt = ChatPromptTemplate.from_messages(        [            ("system", system_prompt),            MessagesPlaceholder(variable_name="messages"),            (                "system",                "Given the conversation above, who should act next?"                " Or should we FINISH? Select one of: {options}",            ),        ]    ).partial(options=str(options), team_members=", ".join(members))        return (        prompt        | llm.bind_functions(functions=[function_def], function_call="route")        | JsonOutputFunctionsParser()    )from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholderfrom langchain_core.messages import HumanMessage, SystemMessagefrom typing import List, Dict, Callable, Uniondef create_team_supervisor2(llm, system_prompt: str, members: List[str]) -> Callable:    """An LLM-based router compatible with Claude."""    options = ["FINISH"] + members        prompt = ChatPromptTemplate.from_messages(        [            ("system", system_prompt),            MessagesPlaceholder(variable_name="chat_history"),            (                "human",                "Given the conversation above, who should act next?"                " Or should we FINISH? Select one of: {options}"                " Respond with only the name of the next actor or FINISH, nothing else."            ),        ]    ).partial(options=str(options), team_members=", ".join(members))    def route(state: Dict[str, Union[List[Dict], str, None]]) -> Dict[str, str]:        # Extract the messages from the state        messages = state.get('messages', [])                # Convert dictionary messages to BaseMessage objects if necessary        chat_history = []        for msg in messages:            # print("************msg: ",msg)            if isinstance(msg, dict):                # print("************msg.get('type'): ",msg.get('type'))                if msg.get('type') == 'human':                    chat_history.append(HumanMessage(content=msg['content']))                elif msg.get('type') == 'ai':                    chat_history.append(SystemMessage(content=msg['content']))                    # print("************msg['content']: ",msg['content'])            else:                chat_history.append(msg)        # print("************prompt: ",prompt)        # Format the prompt with the chat history        formatted_prompt = prompt.format_messages(chat_history=chat_history[-1:])                # Invoke the LLM        # print(f"=====create_team_supervisor2>formatted_prompt: ",formatted_prompt)        response = llm.invoke(formatted_prompt)        next_actor = response.content.strip()                if next_actor not in options:            print(f"Invalid response from AI: {next_actor}. Defaulting to FINISH.")            next_actor = "FINISH"                return {"next": next_actor}    return routeteam_members_dict = {        # "Schema_Analyst": schema_node,        # "SQL_Expert": sql_node,         # "Search_Agent": search_node,        "Data_Scientist": data_scientist_node,}team_members = list(team_members_dict.keys())supervisor_agent = create_team_supervisor2(    llm,    "You are a supervisor tasked with managing a conversation between the"    f" following workers: {', '.join(team_members)}."     "Usually, in order to search a dataframe, you will need to analyze the schema, generate a SQL query, and then search the dataframe."    "Given the following user request,"    " respond with the worker to act next. Each worker will perform a"    " task and respond with their results and status. When finished,"    " respond with FINISH.",    team_members,)

In [7]:
# workflow = StateGraph(AgentState)# workflow.add_node("Schema_Analyst", schema_node)# workflow.add_node("SQL_Expert", sql_node)# workflow.add_node("Data_Scientist", search_node)# workflow.add_node("supervisor", supervisor_agent)# workflow.add_edge("Schema_Analyst", "supervisor")# workflow.add_edge("SQL_Expert", "supervisor")# workflow.add_edge("Data_Scientist", "supervisor")workflow = StateGraph(AgentState)workflow.add_node("supervisor", supervisor_agent)for agt, nd in team_members_dict.items():    workflow.add_node(agt, nd)    workflow.add_edge(agt, "supervisor")workflow.add_conditional_edges(    "supervisor",    lambda x: x["next"],    dict(        **{d:d for d in team_members},        **{"FINISH": END}        ),)workflow.add_edge(START, "supervisor")graph = workflow.compile()initial_state = AgentState(    messages=[HumanMessage(content="Search the latest close price of symbol '000100.KS' from the dataframe in the path I give you.", name="Human")],    # messages=[HumanMessage(content="Search the latest close price of symbol '000100.KS'. You need to proceed as the process 1) find the schema of the dataframe, 2) generate a SQL query, and 3) search the dataframe with the SQL query from 2).", name="Human")],    team_members=team_members,    next=None,    # dataframe_json=None, #df.to_json(orient='records'),    dataframe_path="data/stock_data_krx.csv",    table_name='df',    # sql_query=None,    # search_result=None,    agent_result="")# llm = ChatOpenAI(model="gpt-3.5-turbo")# recursion_limit = 5# for output in graph.stream(initial_state, config={"recursion_limit": recursion_limit}):#     print("Workflow output:", output)#     if "messages" in list(output.values())[-1]:#         print("Assistant:", list(output.values())[-1]['messages'][-1].content)

In [ ]:
graph = workflow.compile()

recursion_limit = 10

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream(AgentState(
        messages=[HumanMessage(content=user_input, name="Human")],
        # messages=[HumanMessage(content="Search the latest close price of symbol '000100.KS'. You need to proceed as the process 1) find the schema of the dataframe, 2) generate a SQL query, and 3) search the dataframe with the SQL query from 2).", name="Human")],
        team_members=team_members,
        next=None,
        # dataframe_json=None, #df.to_json(orient='records'),
        dataframe_path="data/stock_data_krx.csv",
        table_name='df',
        # sql_query=None,
        # search_result=None,
        agent_result=""
    ), config={"recursion_limit": recursion_limit}):
        # print("Workflow output:", event.values())
        for value in event.values():
            if "messages" in value:
                # print("Assistant:", value['messages'][-1].content)
                value['messages'][-1].pretty_print()